In [ ]:
#Omid55
import requests
import pandas as pd
import pickle as pk
import csv
import time
import numpy as np
import random
from collections import defaultdict
import seaborn as sns
import matplotlib.pylab as plt
from scipy.sparse import csr_matrix
import networkx as nx
import glob
import networkx
import scipy
import classification_pipeline as pi
from __future__ import print_function
% matplotlib inline

In [ ]:
"""Brief and useful info Match class"""
class Match:
    def __init__(self):
        self.summonerIds_championIds_isWinner_tier = []
        self.matchCreation = 0
        self.matchDuration = 0
        self.matchId = 0
        self.matchMode = ""
        self.matchType = ""
        
    def __init__(self, match):
        self.summonerIds_championIds_isWinner_tier = get_summonerIds_championIds_isWinner_tier(match)
        self.matchCreation = match['matchCreation']
        self.matchDuration = match['matchDuration']
        self.matchId = match['matchId']
        self.matchMode = match['matchMode']
        self.matchType = match['matchType']
        
    def __str__(self):
        return 'id:' + str(self.matchId) + ', mode:' + str(self.matchMode) + ', type:' + str(self.matchType) + ', creation:' + str(self.matchCreation) + ', duration:' + str(self.matchDuration) + ', my stats:' + str(self.summonerIds_championIds_isWinner_tier)
    

In [ ]:
class MatchResult:
    def __init__(self):
        self.matchId = 0
        self.won = False
    def __init__(self, matchId, won):
        self.matchId = matchId
        self.won = won

In [ ]:
"""ChampionResult is a simple class with number of
    wins and loses as integer value"""
class ChampionResult:
    def __init__(self):
        self.wins = 0
        self.loses = 0
    
    def get_win_ratio(self):
        #assert self.wins + self.loses != 0, 'There is no matches available...'
        if not self.wins + self.loses:
            return 0
        return self.wins / float(self.wins + self.loses)
    
    def get_game_count(self):
        return self.wins + self.loses
        
        
"""SummonerChampionStats has a hashtable of champions which
    describes how many time summoner won or lost with that
    (GameResult as each item)"""
class SummonerChampionStats:
    def __init__(self):
        self.champions = defaultdict(lambda: ChampionResult())
    
    def get_game_count(self):
        return sum([c.get_game_count() for c in self.champions.values()])
    
    def get_win_ratio(self):
        w = sum([c.wins for c in self.champions.values()])
        l = sum([c.loses for c in self.champions.values()])
        #assert w + l != 0, 'There is no matches available...'
        if not w + l:
            return 0
        return w / float(w+l)
        
    def get_same_champion_game_count(self, champId):
        return self.champions[champId].get_game_count()
        
    def get_same_champion_win_ratio(self, champId):
        return self.champions[champId].get_win_ratio()
        

In [ ]:
# build a key for each match from the summoner Ids:
#   basically first we sort their sids and then put 
#   their ids together with commas
def get_key_of_sids_for_the_given_match(match):
    winner_sids = []
    loser_sids = []
    for player in match.summonerIds_championIds_isWinner_tier:
        if player[2]: 
            winner_sids.append(player[0])
        else:
            loser_sids.append(player[0])
    winner_sids = sorted(winner_sids)
    winner_key = str(winner_sids[0])
    for s in winner_sids[1:]:
        winner_key = winner_key + ',' + str(s)
    loser_sids = sorted(loser_sids)
    loser_key = str(loser_sids[0])
    for s in loser_sids[1:]:
        loser_key = loser_key + ',' + str(s)
    return winner_key, loser_key

# Loading

In [ ]:
# loading all saved matches
matches = {}
for fname in glob.glob("MATCHES_till_sid*.pkl"):
    print(fname)
    with open(fname, 'rb') as f:
        matches_chunk = pk.load(f)
        # it also removes identical matches (with the same matchId) and store them as one match
        matches = dict(matches.items()+matches_chunk.items())   # adding all matches together
print('Length of all matches is', len(matches))

maximum number (sid) in the data

In [ ]:
sids = []
for match in matches.values():
    sids+=[m[0] for m in match.summonerIds_championIds_isWinner_tier]
print(max(sids))

In [ ]:
plt.hist(sids)

looking at all games that a team plays (all same players)

In [ ]:
teams = defaultdict(list)
for k, mat in matches.items():
    w, l = get_key_of_sids_for_the_given_match(mat)
    teams[w].append( MatchResult(mat.matchId, True) )
    teams[l].append( MatchResult(mat.matchId, False) )

removing those teams which play fewer than a certain threshold (MIN_NUMBER_OF_GAMES_TOGHETHER)

In [ ]:
MIN_NUMBER_OF_GAMES_TOGHETHER = 10
to_be_deleted = []
for k in teams.keys():
    if len(teams[k]) < MIN_NUMBER_OF_GAMES_TOGHETHER:
        to_be_deleted.append(k)
print('All teams#: ', len(teams))
for k in to_be_deleted:
    del(teams[k])
print('Now teams#: ', len(teams))

In [ ]:
# sorting matches based on their time of match creation
for k in teams.keys():
    c = []
    for kk in range(len(teams[k])):
        c.append(matches[teams[k][kk].matchId].matchCreation)
    teams[k] = [x for (y, x) in sorted(zip(c, teams[k]))]

the distribution of number of games in the selected teams

In [ ]:
c = []
for k in teams.keys():
    c.append(len(teams[k]))
plt.hist(c)

In [ ]:
diff = []
for k in teams.keys():
    s = []
    for game_result in teams[k]:
        if game_result.won:
            s.append(1)
        else:
            s.append(0)
    diff.append(sum(s[len(s)/2:]) - sum(s[:len(s)/2]))
diff = np.array(diff)

In [ ]:
plt.hist(diff)

In [ ]:
len(np.where(diff>0)[0])

In [ ]:
len(np.where(diff<0)[0])

In [ ]:
len(np.where(diff==0)[0])

# Getting more information about matches

In [ ]:
import download_data
reload(download_data)
api = download_data.get_the_api(key_file_name='MyKey6.key')

In [ ]:
for team, team_matches in teams.items():
    for match_result in team_matches:
        match = api._get_match(match_result.matchId)
        print(match)
        break
    break

In [ ]:
match.keys()

In [ ]:
type(match)

In [ ]:
match['participants'][0]['stats']